In [1]:
from scipy.special import gammaln, betaln, psi
from scipy.optimize import check_grad
from scipy.optimize import minimize, approx_fprime
from math_special_functions import multibetaln, dibetaln, dimultibetaln
import numpy as np

In [2]:
class ShmKmerLikelihood:
    def __init__(self, sample, nonmutated_ind, check_gradient=False, number_of_tests=None):
        self.full_sample = sample
        self.mutated_sample = np.delete(sample, nonmutated_ind, 1)
        self.nonmutated_ind = nonmutated_ind
        self.n_all = np.sum(self.full_sample, axis=1)
        self.n_mutated = np.sum(self.mutated_sample, axis=1)
        if check_gradient:
            if number_of_tests is None:
                print("Supply number of tests for check_gradient.")
            else:
                self.__check_gradient(number_of_tests)
    
    def likelihood(self, beta_shape1, beta_shape2, dir_lambda):
        result_p11 = betaln(beta_shape1 + self.n_mutated,
                            beta_shape2 + self.n_all - self.n_mutated)
        result_p12 = betaln(beta_shape1, beta_shape2)
        
        result_p21 = multibetaln(self.mutated_sample + dir_lambda, 1)
        result_p22 = multibetaln(dir_lambda)
            
        return np.sum(result_p11 - result_p12 + result_p21 - result_p22)
    
    def gradient(self, beta_shape1, beta_shape2, dir_lambda):
        N = self.full_sample.shape[0]
        result_p11 = np.sum(dibetaln(beta_shape1 + self.n_mutated,
                           beta_shape2 + self.n_all - self.n_mutated), 0)
        result_p12 = dibetaln(beta_shape1, beta_shape2).flatten() * N

        result_p21 = np.sum(dimultibetaln(self.mutated_sample + dir_lambda), 0)
        result_p22 = dimultibetaln(dir_lambda) * N
    
        return np.concatenate((result_p11 - result_p12,
                               result_p21 - result_p22))
    
    def __check_gradient(self, number_of_tests=None):
        cglikelihood = (lambda x: self.likelihood(beta_shape1=x[0], beta_shape2=x[1], dir_lambda=x[2:]))
        cggrad       = (lambda x: self.gradient(  beta_shape1=x[0], beta_shape2=x[1], dir_lambda=x[2:]))
        if number_of_tests is not None:
            for i in xrange(number_of_tests):
                point = np.random.exponential(size=5, scale=10)
                print(check_grad(cglikelihood, cggrad, x0=point))
        if x is not None:
            print(check_grad(cglikelihood, cggrad, x))

In [3]:
class ShmKmerLikelihoodOptimizator:
    def __init__(self, shm_kmer_likelihood, x0=None, bounds=None, method='L-BFGS-B'):
        if x0 is None:
            self.x0 = self.__get_start_point(shm_kmer_likelihood)
        if bounds is None:
            self.bounds=((0, None),) * (2 + shm_kmer_likelihood.mutated_sample.shape[1])
        self.method=method
        self.lkhd = (lambda x: -shm_kmer_likelihood.likelihood(beta_shape1=x[0],
                                                               beta_shape2=x[1],
                                                               dir_lambda=x[2:]))
        self.grad = (lambda x: -shm_kmer_likelihood.gradient(beta_shape1=x[0],
                                                             beta_shape2=x[1],
                                                             dir_lambda=x[2:]))
        
    def __get_start_point(self, shm_kmer_likelihood, scale_beta=1, scale_dir=1):
        beta_shape1 = np.mean(1 - shm_kmer_likelihood.full_sample[:,shm_kmer_likelihood.nonmutated_ind] / \
                              np.sum(shm_kmer_likelihood.full_sample, axis=1, dtype=float))
        beta_shape2 = 1 - beta_shape1
        beta_shape1 *= scale_beta
        beta_shape2 *= scale_beta
        
        scaled_mutated_sample = (shm_kmer_likelihood.mutated_sample.T / \
                                 np.sum(shm_kmer_likelihood.mutated_sample, axis=1, dtype=float)).T
        dir_lambda = np.mean(scaled_mutated_sample, axis=0, dtype=float)
        dir_lambda *= scale_dir
        return np.concatenate([np.array([beta_shape1, beta_shape2]), dir_lambda])
    
    def maximize(self):
        minimize_result = minimize(fun=self.lkhd,
                                   x0=self.x0,
                                   bounds=self.bounds,
                                   method=self.method,
                                   jac=self.grad)
        return minimize_result.x

In [4]:
def generate_sample(sample_size, number_samples,
                    real_beta_shape1, real_beta_shape2, real_dir_lambda,
                    nonmutated_ind):
    mutation_bin_prob = np.random.beta(real_beta_shape1, real_beta_shape2, size=number_samples)
    is_mutated = np.random.binomial(sample_size, p=mutation_bin_prob, size=number_samples)

    mutation_dir_probs = np.random.dirichlet(real_dir_lambda, size=number_samples)
    sample = []
    for n, pvals in zip(is_mutated, mutation_dir_probs):
        sample.append(np.random.multinomial(n=n, pvals=pvals, size=1))
    sample = np.array(sample).reshape((number_samples, len(real_dir_lambda)))
    final_sample = np.insert(sample, nonmutated_ind, sample_size - is_mutated, axis=1)
    return final_sample

In [5]:
sample_nonmutated_ind = 3

In [15]:
sample = generate_sample(sample_size=10000, number_samples=1000,
                         nonmutated_ind=sample_nonmutated_ind,
                         real_beta_shape1=3,
                         real_beta_shape2=1,
                         real_dir_lambda=np.array([10, 12, 13]))

In [16]:
lkho = ShmKmerLikelihood(sample, sample_nonmutated_ind)

In [17]:
ShmKmerLikelihoodOptimizator(lkho).maximize()

array([  3.33900193,   1.0841127 ,  10.30593461,  12.15386252,  13.32963033])